# Introduction to atoMEC

For all calculations in the paper, we have used the open source average-atom code atoMEC. It can be downloaded
[here](https://github.com/atomec-project/atoMEC).

In this section, we go through the basics of setting up and then running a calcuation in atoMEC.

We first create an `Atom` object which houses the key physical information about the system we want to study, i.e. the temperature and mass density. We start with Aluminium at room temperature:

In [ ]:
# import the Atom object
from atoMEC import Atom

# set up the Aluminium atom
Al_atom = Atom("Al", 300, density=2.7, units_temp="K")

We see this prints some key information about our system, including for example the ionic coupling and electron degeneracy parameters, which are important in warm dense matter (WDM). For details of how these are computed see our initial [preprint](https://arxiv.org/abs/2103.09928) on AA models and documentation in the code.

Next we set up a `model` object, which contains input regarding which approximations we use in our model, for example the boundary condition and exchange-correlation (XC) approximation.

In [ ]:
# import models
from atoMEC import models

# set up the ISModel
Al_model = models.ISModel(
    Al_atom, bc="dirichlet", unbound="quantum", xfunc_id="lda_x", cfunc_id="lda_c_pw"
)

In the above, we have set up an ion-sphere model (`ISModel`) which so far is the only kind of AA model implemented in atoMEC. Furthermore, we have specified the following approximations:

* `unbound="quantum"`: This means all our KS orbitals are treated in the same way, regardless of their energy
* `bc="dirichlet"`: The Dirichlet boundary condition (as described in the main text) is applied to the orbitals
* `xfunc_id="lda_x"`, `cfunc_id="lda_c_pw"`: We have chosen the LDA XC functional 

We are now ready to run an SCF calculation, which is done by the `CalcEnergy` function. There are various inputs to this function which control numerical aspects, such as the number of grid points and SCF convergence parameters. Most of these are optional so we use the default values for now.

The two parameters which must be specified are the maximal value of the principal and angular quantum numbers, `nmax` and `lmax`. atoMEC will search for all the eigenvalues in the range $0<n<\textrm{nmax}$, $0<l<\textrm{lmax}$. Since we have a system at room temperature we do not need to include lots of states so we set `nmax=5`, `lmax=3`.

In [ ]:
# set the values of nmax and lmax
nmax = 5
lmax = 3

# run the SCF calculation
output = Al_model.CalcEnergy(nmax, lmax, scf_params={"mixfrac": 0.6, "maxscf": 50})

In the above, at each step of the SCF (self-consistent field) cycle, the spherically symmetric KS equations are solved for the chosen boundary condition (Eq. (2) of the main paper). In atoMEC, we solve the KS equations on a logarithmic grid to give more weight to the points nearest the origin, i.e. $x=\log(r)$. Furthermore, we make a transformation of the orbitals $P_{nl}(x) = X_{nl}(x)\exp(x/2)$. Then the equations to be solve become:

\begin{gather}
\frac{\textrm{d}^2 P_{nl}(x)}{\textrm{d}x^2} - 2e^{2x}(W(x)-\epsilon_{nl})P_{nl}(x)=0\,,\\
W(x) = v_\textrm{s}[n](x) + \frac{1}{2}\left(l+\frac{1}{2}\right)^2 e^{-2x}
\end{gather}

In atoMEC, we solve the KS equations using a matrix implementation of Numerov's algorithm as described in [this paper](https://aapt.scitation.org/doi/full/10.1119/1.4748813?casa_token=UMs6bxc3iB0AAAAA%3AonvjnFq-KyEXZpEzUfGfyqQoNrMoP6AI0Wi7nrZrILOCM9Ah55XACGen5VLr-civFUtr2sVuCpw). This means we diagonalize the following equation:
\begin{align}
\hat{H}\vec{P} &= \vec{\epsilon} \hat{B} \vec{P} \\
\hat{H} &= \hat{T} + \hat{B} + W_\textrm{s}(\vec{x}) \\
\hat{T} &= -\frac{1}{2} e^{-2\vec{x}} \hat{A} \\
\hat{A} &= \frac{\hat{I}_{-1} -2\hat{I}_0 + \hat{I}_1}{\textrm{d}x^2} \\
\hat{B} &= \frac{\hat{I}_{-1} +10\hat{I}_0 + \hat{I}_1}{12}\,,
\end{align}
where $\hat{I}_{-1/0/1}$ are lower shift, identify and upper shift matrices.

Since the Hamiltonian matrix $H$ is sparse and we only seek the lowest lying eigenvalues, there is no need to perform a full diagonalization which scales with $\mathcal{O}(N^3)$, with $N$ being the size of the radial grid. Instead, we use a sparse matrix diagonalization routine from the [SciPy](https://scipy.org/) library, which scales more efficiently and allows us to go to larger grid sizes (and hence better convergence).

After each step in the SCF cycle, the relative changes in the free energy $F$, density $n$ and potential $v_\textrm{s}$ are computed. Specifically, the quantities computed are

\begin{align}
    \Delta F &= \left|\frac{F^{i}-F^{i-1}}{F^{i}}\right| \\
    \Delta n &= \frac{\int \mathrm{d}r|n^i(r)-n^{i-1}(r)|}{\int \mathrm{d}r n^i(r)}\\
    \Delta v &= \frac{\int \mathrm{d}r|v^i_\textrm{s}(r)-v_\textrm{s}^{i-1}(r)|}{\int \mathrm{d}r v_\textrm{s}^i(r)}
\end{align}

The values of these convergence parameters are controlled by the input parameter `conv_params` to the `CalcEnergy` function. For example, if we wanted to reduce the required convergence, we could call:

`output=Al_model.CalcEnergy(nmax, lmax, conv_params={"econv": 1e-4, "nconv": 1e-3, "vconv": 1e-3})`

At each stage of the SCF cycle, the KS potential is mixed with some fraction of the KS potential from the previous iteration (which aids convergence), i.e. $v_\textrm{s}(r) = \alpha v^{i+1}_\textrm{s}(r) + (1-\alpha) v^i_\textrm{s}(r)$, where $\alpha$ is the mixing paramater. As seen above, this is controlled via the `scf_params` input parameter to the `CalcEnergy` function. This dictionary also accepts a `maxscf` key to terminate the SCF cycle after the requested number of iterations.

At the end of the SCF cycle, various information is printed, such as the breakdown of the total free energy and the KS eigenvalues and their occupations. The printed ''Mean ionization state'' (MIS) output is calculated using the threshold method described in the main paper. We shall later see how to compute the MIS in atoMEC via the other methods described in the paper. The output of the `CalcEnergy` function is a dictionary containing information about the energy, density, potential and orbtials. For example, we can extract and plot the density from this output:

In [ ]:
# import matplotlib for plotting and numpy for analysis
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# extract the density object
dens_Al = output["density"]

# get the total density and grid
dens_tot = dens_Al.total[0]
xgrid = dens_Al._xgrid  # log grid
rgrid = np.exp(xgrid)  # radial grid

# plot the total density
plt.plot(rgrid, rgrid ** 2 * dens_tot)

# some formatting
plt.xlim(0,3)
plt.ylim(0,1.2)
plt.xlabel(r'$a_0$')
plt.ylabel(r'$r^2 n(r)\ (a_0)^{-1}$')
plt.show()

## Convergence testing

There are various parameters which should be checked for convergence. For all the boundary conditions, the main ones are:

* `nmax`: the maximum number of the principal quantum number `n`
* `lmax`: the maximum number of the angular quantum number `l`
* `grid_params`: dictionary parameter controlling the logarithmic grid, in particular the number of grid points `ngrid`

Furthermore, for the `bands` boundary condition, there is the additional dictionary parameter `band_params`. The important property within this is `nkpts` number of 'k' points, which is the number of states that are computed within all energy bands (spaced linearly in energy) in our model.

The convergence for the `nmax` and `lmax` paramaters can generally be chosen by eye, in other words by ensuring there are sufficient states such that the highest levels have (nearly) zero occoupations. Let us consider our Aluminium atom from earlier, but now we shall increase the temperature (so more orbitals are required).

In [ ]:
# set the temperature to 10 eV
Al_atom.units_temp="eV"
Al_atom.temp = 10

# run the calculation again
output = Al_model.CalcEnergy(nmax, lmax, write_info=True)

In the above example, we see that `nmax=5` seems to be sufficiently large, but `lmax=2` is not, because the occupation of the $l=2,n=0$ state is 0.834, i.e. significantly above zero. We therefore choose a much larger value of `lmax` to check what is required for convergence. 

Since the diagonalization must be performed separately for every value of $0<l<\textrm{lmax}$, the computational time is proportional to `lmax`. There is a simple kind of parallelization implemented in atoMEC, courtesy of the [joblib](https://joblib.readthedocs.io/en/latest/#) library, which parallelizes the calculation over `l` (and also spin if `model.spinpol=True`) and thus makes calculations more efficient. This is enabled by the `config.numcores` parameter. Setting `config.numcores=n` uses `n` cores; however, it is often easiest to set `config.numcores=-1` which uses all the available cores.

In [ ]:
#enable parallelization
from atoMEC import config
config.numcores = -1

# re-run the calculation with larger lmax
lmax = 10
output = Al_model.CalcEnergy(nmax, lmax)